<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Futures_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [86]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/final_BANKNIFTY_F2.csv")
data['Date'] = pd.to_datetime(data['Date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') # Converted to reqd format
data.sort_values(by='Date', inplace=True)
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time
start_time = pd.to_datetime('09:15:00', format='%H:%M:%S').time()
end_time = pd.to_datetime('15:30:00', format='%H:%M:%S').time()
filtered_data = data[(data['Time'] >= start_time) & (data['Time'] <= end_time)] #Filtered out the data that doesnt occur in the time range
filtered_data

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
813167,BANKNIFTY_F2,01-01-2018,15:20:00,25515.20,25515.20,25515.20,25515.20,80.0,NaN
809216,BANKNIFTY_F2,01-01-2018,12:38:00,25637.50,25637.50,25637.50,25637.50,40.0,NaN
809217,BANKNIFTY_F2,01-01-2018,12:39:00,25640.00,25640.00,25640.00,25640.00,40.0,NaN
809218,BANKNIFTY_F2,01-01-2018,13:20:00,25635.80,25635.80,25625.10,25625.10,120.0,NaN
809219,BANKNIFTY_F2,01-01-2018,13:24:00,25626.10,25626.10,25625.10,25625.10,120.0,NaN
...,...,...,...,...,...,...,...,...,...
562680,BANKNIFTY_F2,31-12-2021,10:42:00,35766.75,35769.00,35766.75,35769.00,325.0,109775.0
562681,BANKNIFTY_F2,31-12-2021,10:44:00,35775.00,35775.00,35765.00,35765.00,75.0,109775.0
562682,BANKNIFTY_F2,31-12-2021,10:45:00,35760.00,35772.55,35760.00,35772.55,75.0,109775.0
562683,BANKNIFTY_F2,31-12-2021,10:48:00,35766.80,35766.80,35766.80,35766.80,100.0,109750.0


In [87]:
import pandas as pd
df = pd.DataFrame(filtered_data)
df['Open Interest'] = df['Open Interest'].fillna(0) #Replaced Nan by 0
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'].astype(str), format='%d-%m-%Y %H:%M:%S', errors='coerce') #Concatenated Date and Time to set a unique index
df = df.set_index('DateTime')
df = df.dropna(subset=['Date', 'Time'])
all_days = df.index.normalize().unique()
full_range = pd.concat([pd.DataFrame(index=pd.date_range(start=f"{day} 09:16", end=f"{day} 15:30", freq='1T')) for day in all_days])
full_range = full_range.merge(df, how='left', left_index=True, right_index=True)
full_range.fillna(method='ffill', inplace=True)
full_range.reset_index(inplace=True)
full_range.rename(columns={'index': 'DateTime'}, inplace=True)
full_range['Date'] = full_range['DateTime'].dt.strftime('%d-%m-%Y')
full_range['Time'] = full_range['DateTime'].dt.strftime('%H:%M:%S')
full_range = full_range[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]
full_range['DateTime'] = pd.to_datetime(full_range['Date'] + ' ' + full_range['Time'], format='%d-%m-%Y %H:%M:%S')
full_range.set_index('DateTime', inplace=True)

aggregation = {
    'Symbol': 'first',
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Open Interest': 'last'
}

resampled_df = full_range.resample('5T').apply(aggregation)

# Reset index to split DateTime into Date and Time
resampled_df.reset_index(inplace=True)
resampled_df['Date'] = resampled_df['DateTime'].dt.strftime('%d-%m-%Y')
resampled_df['Time'] = resampled_df['DateTime'].dt.strftime('%H:%M:%S')

resampled_df.drop(columns=['DateTime'], inplace=True)
resampled_df = resampled_df[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest']]

resampled_df


,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0
...,...,...,...,...,...,...,...,...,...
542663,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0
542664,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0
542665,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0
542666,BANKNIFTY_F2,28-02-2023,15:25:00,40715.20,40715.20,40694.1,40706.00,4700.0,1173225.0


In [88]:
resampled_df.head(60)

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.0,25643.1,25608.0,25643.1,480.0,0.0
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.5,25675.7,25631.5,25665.3,600.0,0.0
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.3,25665.3,25655.1,25655.1,240.0,0.0
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.7,25647.7,25647.7,25647.7,240.0,0.0
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.7,25647.7,25647.0,25647.0,440.0,0.0
5,BANKNIFTY_F2,01-01-2018,09:40:00,25655.1,25661.9,25655.0,25655.0,320.0,0.0
6,BANKNIFTY_F2,01-01-2018,09:45:00,25653.0,25653.0,25645.0,25650.0,440.0,0.0
7,BANKNIFTY_F2,01-01-2018,09:50:00,25645.0,25650.0,25645.0,25650.0,400.0,0.0
8,BANKNIFTY_F2,01-01-2018,09:55:00,25645.0,25650.0,25645.0,25650.0,400.0,0.0
9,BANKNIFTY_F2,01-01-2018,10:00:00,25642.0,25642.0,25642.0,25642.0,1680.0,0.0


In [89]:
df = pd.DataFrame(resampled_df)
df

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0
...,...,...,...,...,...,...,...,...,...
542663,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0
542664,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0
542665,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0
542666,BANKNIFTY_F2,28-02-2023,15:25:00,40715.20,40715.20,40694.1,40706.00,4700.0,1173225.0


In [90]:

df['DateTime'] = pd.to_datetime(df['Date'].str.strip() + ' ' + df['Time'].str.strip(), format='%d-%m-%Y %H:%M:%S', errors='coerce')
df = df.dropna(subset=['DateTime'])  # Remove rows where DateTime could not be parsed
df.set_index('DateTime', inplace=True)
# Calculate typical price for VWAP
df.loc[:, 'Price'] = (df['High'] + df['Low'] + df['Close']) / 3

# Calculate cumulative values for VWAP
cumulative_price_volume = (df['Price'] * df['Volume']).cumsum()
cumulative_volume = df['Volume'].cumsum()

df.loc[:, 'VWAP'] = cumulative_price_volume / cumulative_volume #Calculating for 1 min

# Aggregate the data into 5-minute intervals
resampled = df.resample('5T').agg({
    'Symbol': 'first',
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'Open Interest': 'last',
    'Price': 'last'
})
# Calculate cumulative values for the resampled data
resampled['Price'] = (resampled['High'] + resampled['Low'] + resampled['Close']) / 3
resampled_cumulative_price_volume = (resampled['Price'] * resampled['Volume']).cumsum()
resampled_cumulative_volume = resampled['Volume'].cumsum()

# Calculating VWAP for the 5-minute intervals
resampled['VWAP'] = resampled_cumulative_price_volume / resampled_cumulative_volume

resampled.reset_index(inplace=True)
resampled['Date'] = resampled['DateTime'].dt.strftime('%d-%m-%Y')
resampled['Time'] = resampled['DateTime'].dt.strftime('%H:%M')

resampled.drop(columns=['DateTime', 'Price'], inplace=True) # Dropped DateTime column and unnecessary columns
resampled = resampled[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest', 'VWAP']]

resampled.dropna(inplace=True)
resampled.head(60)


,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP
0,BANKNIFTY_F2,01-01-2018,09:15,25634.0,25643.1,25608.0,25643.1,480.0,0.0,25631.400000
1,BANKNIFTY_F2,01-01-2018,09:20,25631.5,25675.7,25631.5,25665.3,600.0,0.0,25645.900000
2,BANKNIFTY_F2,01-01-2018,09:25,25665.3,25665.3,25655.1,25655.1,240.0,0.0,25648.190909
3,BANKNIFTY_F2,01-01-2018,09:30,25647.7,25647.7,25647.7,25647.7,240.0,0.0,25648.115385
4,BANKNIFTY_F2,01-01-2018,09:35,25647.7,25647.7,25647.0,25647.0,440.0,0.0,25647.921333
5,BANKNIFTY_F2,01-01-2018,09:40,25655.1,25661.9,25655.0,25655.0,320.0,0.0,25649.214943
6,BANKNIFTY_F2,01-01-2018,09:45,25653.0,25653.0,25645.0,25650.0,440.0,0.0,25649.233816
7,BANKNIFTY_F2,01-01-2018,09:50,25645.0,25650.0,25645.0,25650.0,400.0,0.0,25649.119831
8,BANKNIFTY_F2,01-01-2018,09:55,25645.0,25650.0,25645.0,25650.0,400.0,0.0,25649.031461
9,BANKNIFTY_F2,01-01-2018,10:00,25642.0,25642.0,25642.0,25642.0,1680.0,0.0,25646.777099


# MACD

In [91]:
resampled_df_copy = resampled

# Converted 'Date' and 'Time' columns to strings so that we can concatenate
resampled_df_copy['Date'] = resampled_df_copy['Date'].astype(str)
resampled_df_copy['Time'] = resampled_df_copy['Time'].astype(str)
resampled_df_copy['DateTime'] = pd.to_datetime(resampled_df_copy['Date'] + ' ' + resampled_df_copy['Time'], format='%d-%m-%Y %H:%M')

resampled_df_copy = resampled_df_copy.dropna(subset=['DateTime'])
resampled_df_copy.set_index('DateTime', inplace=True) #Setting DateTime column as index

resampled_df_copy['EMA_12'] = resampled_df_copy['Close'].ewm(span=12, adjust=False).mean() #12-period EMA
resampled_df_copy['EMA_26'] = resampled_df_copy['Close'].ewm(span=26, adjust=False).mean() #26-period EMA

resampled_df_copy['MACD'] = resampled_df_copy['EMA_12'] - resampled_df_copy['EMA_26'] # MACD= 12_EMA - 26_EMA

resampled_df_copy.drop(['Date'	,'Time','EMA_12', 'EMA_26'], axis=1, inplace=True)

resampled_df_copy['Date'] = resampled_df_copy.index.strftime('%d-%m-%Y')
resampled_df_copy['Time'] = resampled_df_copy.index.strftime('%H:%M:%S')
resampled_df_copy = resampled_df_copy[['Symbol', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest','VWAP' ,'MACD']]
resampled_df_copy

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,MACD
DateTime,,,,,,,,,,,
2018-01-01 09:15:00,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,25631.400000,0.000000
2018-01-01 09:20:00,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25645.900000,1.770940
2018-01-01 09:25:00,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25648.190909,2.324572
2018-01-01 09:30:00,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25648.115385,2.141525
2018-01-01 09:35:00,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.921333,1.917867
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 15:10:00,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,31839.317288,-3.280036
2023-02-28 15:15:00,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,31839.331858,-1.197626
2023-02-28 15:20:00,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,31839.379226,1.723892


# POC

In [92]:
data = resampled_df_copy.copy()
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%d-%m-%Y %H:%M:%S')
data.reset_index(drop=True, inplace=True)
data.head()

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,MACD,DateTime
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.0,25643.1,25608.0,25643.1,480.0,0.0,25631.400000,0.000000,2018-01-01 09:15:00
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.5,25675.7,25631.5,25665.3,600.0,0.0,25645.900000,1.770940,2018-01-01 09:20:00
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.3,25665.3,25655.1,25655.1,240.0,0.0,25648.190909,2.324572,2018-01-01 09:25:00
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.7,25647.7,25647.7,25647.7,240.0,0.0,25648.115385,2.141525,2018-01-01 09:30:00
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.7,25647.7,25647.0,25647.0,440.0,0.0,25647.921333,1.917867,2018-01-01 09:35:00


In [103]:
df = pd.DataFrame(data)

# Convert DateTime to datetime object
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Extracting the Date Part from 'DateTime'
df['Date'] = df['DateTime'].dt.date

# Calculate POC
aggregated = df.groupby(['Date', 'Close']).agg({'Volume': 'sum'}).reset_index()
poc = aggregated.loc[aggregated.groupby('Date')['Volume'].idxmax()]
poc.rename(columns={'Close': 'Point of Control'}, inplace=True)

# Calculate Local Maximum and Minimum
daily_high_low = df.groupby('Date').agg({'High': 'max', 'Low': 'min'}).reset_index()
daily_high_low.rename(columns={'High': 'Local Maximum', 'Low': 'Local Minimum'}, inplace=True)

# Merge POC and daily high/low back into the original dataframe
df = df.merge(poc[['Date', 'Point of Control']], on='Date', how='left')
df = df.merge(daily_high_low, on='Date', how='left')
df

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,MACD,DateTime,Point of Control,Local Maximum,Local Minimum
0,BANKNIFTY_F2,2018-01-01,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,25631.400000,0.000000,2018-01-01 09:15:00,25659.9,25675.7,25481.3
1,BANKNIFTY_F2,2018-01-01,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25645.900000,1.770940,2018-01-01 09:20:00,25659.9,25675.7,25481.3
2,BANKNIFTY_F2,2018-01-01,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25648.190909,2.324572,2018-01-01 09:25:00,25659.9,25675.7,25481.3
3,BANKNIFTY_F2,2018-01-01,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25648.115385,2.141525,2018-01-01 09:30:00,25659.9,25675.7,25481.3
4,BANKNIFTY_F2,2018-01-01,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.921333,1.917867,2018-01-01 09:35:00,25659.9,25675.7,25481.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96895,BANKNIFTY_F2,2023-02-28,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,31839.317288,-3.280036,2023-02-28 15:10:00,40600.0,40900.0,40513.0
96896,BANKNIFTY_F2,2023-02-28,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,31839.331858,-1.197626,2023-02-28 15:15:00,40600.0,40900.0,40513.0
96897,BANKNIFTY_F2,2023-02-28,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,31839.379226,1.723892,2023-02-28 15:20:00,40600.0,40900.0,40513.0
96898,BANKNIFTY_F2,2023-02-28,15:25:00,40715.20,40715.20,40694.1,40706.00,4700.0,1173225.0,31839.416324,4.228513,2023-02-28 15:25:00,40600.0,40900.0,40513.0


In [ ]:
'''#df_['DateTime'] = pd.to_datetime(df_['Date'] + ' ' + df_['Time'], format='%d-%m-%Y %H:%M:%S')
df.set_index('DateTime', inplace=True)

def calculate_vah_val_poc(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume  # Here I have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None
    POC = None

    # Accumulate volume and find VAH and VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold and VAL is None:
            VAL = row['midpoint']
        if POC is None or row['Volume'] > POC:
            POC = row['midpoint']

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']

    # Calculate Value Area Width in percentage terms
    value_area_width = ((VAH - VAL) / VAH) * 100

    return pd.Series({'VAH': VAH, 'VAL': VAL, 'POC': POC, 'Value Area Width (%)': value_area_width})

# Calculate daily VAH, VAL, POC
daily_values = df.groupby(df.index.date).apply(calculate_vah_val_poc)

# Calculate rolling metrics
rolling_3d = daily_values[['VAH', 'VAL', 'POC']].rolling(window=3).mean()
rolling_5d = daily_values[['VAH', 'VAL', 'POC']].rolling(window=5).mean()

# Combine results
result = daily_values.join(rolling_3d, rsuffix='_3d').join(rolling_5d, rsuffix='_5d')
df1 = df.merge(result, left_index=True, right_index=True, how='left')
df1'''

In [95]:
df1['VAH_3d'].unique()

array([nan])

In [104]:
df.set_index('DateTime', inplace=True)

def calculate_vah_val_poc(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume  # 70% volume threshold

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculate the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None
    POC = df_sorted.iloc[0]['midpoint']  # Initialize POC as the highest midpoint

    # Accumulate volume to find VAL and POC
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold:
            VAL = row['midpoint']
            break

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']

    # Calculate Value Area Width in percentage terms
    value_area_width = ((VAH - VAL) / VAH) * 100

    return pd.Series({'VAH': VAH, 'VAL': VAL, 'POC': POC, 'Value Area Width (%)': value_area_width})

# Group by each day and apply the calculate_vah_val_poc function
df_daily = df.groupby(df.index.date).apply(calculate_vah_val_poc)

# Merge the daily results back into the original DataFrame
df = df.join(df_daily, on=df.index.date)

'''# Function to calculate rolling VAH, VAL, and POC for given window size
def calculate_rolling_vah_val_poc(df, window):
    rolling_vah = df['VAH'].rolling(window=window).max()
    rolling_val = df['VAL'].rolling(window=window).min()
    rolling_poc = df['POC'].rolling(window=window).apply(lambda x: x.value_counts().idxmax())
    return rolling_vah, rolling_val, rolling_poc

# Calculate rolling VAH, VAL, and POC for 3-day and 5-day windows
df['VAH_3d'], df['VAL_3d'], df['POC_3d'] = calculate_rolling_vah_val_poc(df_daily, window=3)
df['VAH_5d'], df['VAL_5d'], df['POC_5d'] = calculate_rolling_vah_val_poc(df_daily, window=5)'''
df

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,MACD,Point of Control,Local Maximum,Local Minimum,VAH,VAL,POC,Value Area Width (%)
DateTime,,,,,,,,,,,,,,,,,,
2018-01-01 09:15:00,BANKNIFTY_F2,2018-01-01,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,25631.400000,0.000000,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775
2018-01-01 09:20:00,BANKNIFTY_F2,2018-01-01,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25645.900000,1.770940,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775
2018-01-01 09:25:00,BANKNIFTY_F2,2018-01-01,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25648.190909,2.324572,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775
2018-01-01 09:30:00,BANKNIFTY_F2,2018-01-01,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25648.115385,2.141525,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775
2018-01-01 09:35:00,BANKNIFTY_F2,2018-01-01,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.921333,1.917867,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 15:10:00,BANKNIFTY_F2,2023-02-28,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,31839.317288,-3.280036,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406
2023-02-28 15:15:00,BANKNIFTY_F2,2023-02-28,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,31839.331858,-1.197626,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406
2023-02-28 15:20:00,BANKNIFTY_F2,2023-02-28,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,31839.379226,1.723892,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406


In [105]:
#df = resampled.copy()
q = pd.DataFrame(resampled)

q['DateTime'] = pd.to_datetime(q['Date'] + ' ' + q['Time'], format='%d-%m-%Y %H:%M')
q.set_index('DateTime', inplace=True)

def calculate_vah_val(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume #Here i have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2   # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None

    # Accumulate volume and find VAH and VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold:
            VAL = row['midpoint']
            break

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']
    return pd.Series({'VAH': VAH, 'VAL': VAL})

result = q.groupby(df.index.date).apply(calculate_vah_val)
result

,VAH,VAL
2018-01-01,25660.200,25545.300
2018-01-02,25529.000,25422.450
2018-01-03,25544.500,25488.900
2018-01-04,25565.700,25510.000
2018-01-05,25711.500,25624.950
...,...,...
2023-02-22,40788.050,40354.125
2023-02-23,40392.800,40189.025
2023-02-24,40783.300,40425.000
2023-02-27,40805.650,40558.500


In [135]:
import pandas as pd

# Assuming 'result' DataFrame is already defined
# And original dataframe 'df' is defined

# Function to calculate ghost zone for each day
def calculate_ghost_zone(df_day, vah, val):
    # Filter prices above VAH and below VAL
    above_vah = df_day[df_day['High'] > vah]
    below_val = df_day[df_day['Low'] < val]

    highest_above_vah = above_vah['High'].max() if not above_vah.empty else None
    lowest_below_val = below_val['Low'].min() if not below_val.empty else None

    return pd.Series({'Highest Above VAH': highest_above_vah, 'Lowest Below VAL': lowest_below_val})

# Join the original df with the calculated daily VAH and VAL
daily_vah_val = result[['VAH', 'VAL']]

# Iterate over each day to calculate the ghost zone
ghost_zones = []
for date, vah_val in daily_vah_val.iterrows():
    vah = vah_val['VAH']
    val = vah_val['VAL']
    df_day = df[df.index.date == pd.Timestamp(date).date()]

    ghost_zone = calculate_ghost_zone(df_day, vah, val)
    ghost_zones.append(ghost_zone)

# Convert list of series to DataFrame
ghost_zones_df = pd.DataFrame(ghost_zones, index=daily_vah_val.index)

# Combine with the existing result DataFrame
result_with_ghost_zones = result.join(ghost_zones_df)
result_with_ghost_zones


,VAH,VAL,Highest Above VAH,Lowest Below VAL
2018-01-01,25660.200,25545.300,25675.70,25481.30
2018-01-02,25529.000,25422.450,25554.60,25360.00
2018-01-03,25544.500,25488.900,25550.00,25432.80
2018-01-04,25565.700,25510.000,25573.30,25436.30
2018-01-05,25711.500,25624.950,25720.00,25575.00
...,...,...,...,...
2023-02-22,40788.050,40354.125,40850.00,40177.00
2023-02-23,40392.800,40189.025,40437.60,39870.00
2023-02-24,40783.300,40425.000,40841.55,40308.05
2023-02-27,40805.650,40558.500,40815.00,40228.05


In [166]:
#merged_df = df.merge(result_with_ghost_zones[['VAH', 'VAL', 'Highest Above VAH', 'Lowest Below VAL']], how='left', left_on=df.index.date, right_index=True)
merged_df = df.merge(result_with_ghost_zones[['Highest Above VAH', 'Lowest Below VAL']], left_on=df.index.date, right_index=True, how='left')
merged_df

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,MACD,Point of Control,Local Maximum,Local Minimum,VAH,VAL,POC,Value Area Width (%),Highest Above VAH,Lowest Below VAL
DateTime,,,,,,,,,,,,,,,,,,,,
2018-01-01 09:15:00,BANKNIFTY_F2,2018-01-01,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,25631.400000,0.000000,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3
2018-01-01 09:20:00,BANKNIFTY_F2,2018-01-01,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25645.900000,1.770940,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3
2018-01-01 09:25:00,BANKNIFTY_F2,2018-01-01,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25648.190909,2.324572,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3
2018-01-01 09:30:00,BANKNIFTY_F2,2018-01-01,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25648.115385,2.141525,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3
2018-01-01 09:35:00,BANKNIFTY_F2,2018-01-01,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.921333,1.917867,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 15:10:00,BANKNIFTY_F2,2023-02-28,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,31839.317288,-3.280036,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0
2023-02-28 15:15:00,BANKNIFTY_F2,2023-02-28,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,31839.331858,-1.197626,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0
2023-02-28 15:20:00,BANKNIFTY_F2,2023-02-28,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,31839.379226,1.723892,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0


In [167]:
merged_df['Previous Day VAH'] = merged_df['VAH'].shift(1)
merged_df['Previous Day VAL'] = merged_df['VAL'].shift(1)

# Function to determine gap up or gap down
def gap_type(row):
    if row['Open'] >= row['Previous Day VAH']:
        return 'Gap Up'
    elif row['Close'] <= row['Previous Day VAL']:
        return 'Gap Down'
    else:
        return ''

# Apply the function to each row
merged_df['Gap Type'] = merged_df.apply(gap_type, axis=1)

# Handle consecutive missing days
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Date_diff'] = merged_df['Date'].diff().dt.days
merged_df.loc[merged_df['Date_diff'] > 1, 'Gap Type'] = ''

# Group by date and copy-paste any one value present in that day to all the rows of that day
merged_df['Gap Type'] = merged_df.groupby('Date')['Gap Type'].transform(lambda x: x.iloc[0]).fillna('')

# Drop the auxiliary columns if needed
merged_df.drop(['Previous Day VAH', 'Previous Day VAL', 'Date_diff'], axis=1, inplace=True)
merged_df


,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,...,Point of Control,Local Maximum,Local Minimum,VAH,VAL,POC,Value Area Width (%),Highest Above VAH,Lowest Below VAL,Gap Type
DateTime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 09:15:00,BANKNIFTY_F2,2018-01-01,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,25631.400000,...,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,
2018-01-01 09:20:00,BANKNIFTY_F2,2018-01-01,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25645.900000,...,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,
2018-01-01 09:25:00,BANKNIFTY_F2,2018-01-01,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25648.190909,...,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,
2018-01-01 09:30:00,BANKNIFTY_F2,2018-01-01,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25648.115385,...,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,
2018-01-01 09:35:00,BANKNIFTY_F2,2018-01-01,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.921333,...,25659.9,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 15:10:00,BANKNIFTY_F2,2023-02-28,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,31839.317288,...,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0,Gap Up
2023-02-28 15:15:00,BANKNIFTY_F2,2023-02-28,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,31839.331858,...,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0,Gap Up
2023-02-28 15:20:00,BANKNIFTY_F2,2023-02-28,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,31839.379226,...,40600.0,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0,Gap Up


In [ ]:
import pandas as pd

# Assuming merged_df is your DataFrame
# You can use iloc to slice the DataFrame
start_row = 300
end_row = start_row + 60

# Ensure that end_row does not exceed the total number of rows in the DataFrame
if end_row > len(merged_df):
    end_row = len(merged_df)

# Slice the DataFrame to get the desired rows
rows_to_print = merged_df.iloc[start_row:end_row]

# Print the sliced DataFrame
rows_to_print


In [168]:
def is_gap_filled(row):
    if row['Open'] >= row['Previous Day Low'] and row['Open'] <= row['High']:
        return 'Yes'
    else:
        return 'No'

# Calculate the previous day's low (You may have this value available in your DataFrame)
# Assuming you have it, you can calculate it like this:
merged_df['Previous Day Low'] = merged_df['Low'].shift(1)

# Apply the function to each row to determine if the gap is filled
merged_df['Gap Filled'] = merged_df.apply(is_gap_filled, axis=1)

# Drop the auxiliary columns if needed
merged_df.drop('Previous Day Low', axis=1, inplace=True)
merged_df

,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,VWAP,...,Local Maximum,Local Minimum,VAH,VAL,POC,Value Area Width (%),Highest Above VAH,Lowest Below VAL,Gap Type,Gap Filled
DateTime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 09:15:00,BANKNIFTY_F2,2018-01-01,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,25631.400000,...,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,,No
2018-01-01 09:20:00,BANKNIFTY_F2,2018-01-01,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25645.900000,...,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,,Yes
2018-01-01 09:25:00,BANKNIFTY_F2,2018-01-01,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25648.190909,...,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,,Yes
2018-01-01 09:30:00,BANKNIFTY_F2,2018-01-01,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25648.115385,...,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,,No
2018-01-01 09:35:00,BANKNIFTY_F2,2018-01-01,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.921333,...,25675.7,25481.3,25660.200,25545.3,25660.200,0.447775,25675.7,25481.3,,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 15:10:00,BANKNIFTY_F2,2023-02-28,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,31839.317288,...,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0,Gap Up,Yes
2023-02-28 15:15:00,BANKNIFTY_F2,2023-02-28,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,31839.331858,...,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0,Gap Up,Yes
2023-02-28 15:20:00,BANKNIFTY_F2,2023-02-28,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,31839.379226,...,40900.0,40513.0,40813.525,40695.0,40813.525,0.290406,40900.0,40513.0,Gap Up,Yes


In [24]:
'''df = pd.DataFrame(data)

df['DateTime'] = pd.to_datetime(df['DateTime'], format='%d-%m-%Y')
df['Date'] = df['DateTime'].dt.date                     #Extracting the Date Part from 'DateTime'
aggregated = df.groupby(['Date', 'Close']).agg({'Volume': 'sum'}).reset_index()

poc = aggregated.loc[aggregated.groupby('Date')['Volume'].idxmax()] # POC for each date
poc.rename(columns={'Close': 'Point of Control'}, inplace=True)
poc'''

"df = pd.DataFrame(data)\n\ndf['DateTime'] = pd.to_datetime(df['DateTime'], format='%d-%m-%Y')\ndf['Date'] = df['DateTime'].dt.date                     #Extracting the Date Part from 'DateTime'\naggregated = df.groupby(['Date', 'Close']).agg({'Volume': 'sum'}).reset_index()\n\npoc = aggregated.loc[aggregated.groupby('Date')['Volume'].idxmax()] # POC for each date\npoc.rename(columns={'Close': 'Point of Control'}, inplace=True)\npoc"

In [25]:
'''df = pd.DataFrame(data)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Date'] = df['DateTime'].dt.date # Extracting Date from DateTime

daily_high_low = df.groupby('Date').agg({'High': 'max', 'Low': 'min'}).reset_index()
daily_high_low.rename(columns={'High': 'Local Maximum','Low':'Local Minimum'}, inplace=True)
daily_high_low'''

"df = pd.DataFrame(data)\ndf['DateTime'] = pd.to_datetime(df['DateTime'])\ndf['Date'] = df['DateTime'].dt.date # Extracting Date from DateTime\n\ndaily_high_low = df.groupby('Date').agg({'High': 'max', 'Low': 'min'}).reset_index()\ndaily_high_low.rename(columns={'High': 'Local Maximum','Low':'Local Minimum'}, inplace=True)\ndaily_high_low"

In [26]:
'''df = resampled_df.copy()
df = pd.DataFrame(resampled_df)

df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M:%S')
df.set_index('DateTime', inplace=True)

def calculate_vah_val(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume #Here i have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2   # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None

    # Accumulate volume and find VAH and VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold:
            VAL = row['midpoint']
            break

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']
    return pd.Series({'VAH': VAH, 'VAL': VAL})

result = df.groupby(df.index.date).apply(calculate_vah_val)
result'''

"df = resampled_df.copy()\ndf = pd.DataFrame(resampled_df)\n\ndf['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M:%S')\ndf.set_index('DateTime', inplace=True)\n\ndef calculate_vah_val(df_day):\n    total_volume = df_day['Volume'].sum()\n    volume_threshold = 0.7 * total_volume #Here i have considered 70%\n\n    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2   # Calculating the midpoint for each row\n    df_sorted = df_day.sort_values(by='midpoint', ascending=False)\n\n    cumulative_volume = 0\n    VAH = None\n    VAL = None\n\n    # Accumulate volume and find VAH and VAL\n    for index, row in df_sorted.iterrows():\n        cumulative_volume += row['Volume']\n        if cumulative_volume >= volume_threshold:\n            VAL = row['midpoint']\n            break\n\n    # The highest price in the sorted DataFrame is VAH\n    VAH = df_sorted.iloc[0]['midpoint']\n    return pd.Series({'VAH': VAH, 'VAL': VAL})\n\nresult = df.groupby(df.index.da

In [ ]:
'''# Ensure the DateTime column is the index
df.set_index('DateTime', inplace=True)

def calculate_vah_val(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume  # Here I have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None

    # Accumulate volume and find VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold:
            VAL = row['midpoint']
            break

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']

    # Calculate Value Area Width in percentage terms
    value_area_width = ((VAH - VAL) / VAH) * 100

    return pd.Series({'VAH': VAH, 'VAL': VAL, 'Value Area Width (%)': value_area_width})

# Group by each day and apply the calculate_vah_val function
df_daily = df.groupby(df.index.date).apply(calculate_vah_val)

# Merge the results back into the original DataFrame
df1 = df.join(df_daily, on=df.index.date)
df1'''

In [34]:
'''import pandas as pd

# Assuming resampled_df is already defined and contains the dataset
#df = resampled_df.copy()
#df = pd.DataFrame(resampled_df)

#df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M:%S')
df.set_index('DateTime', inplace=True)

def calculate_vah_val(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume  # Here I have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None

    # Accumulate volume and find VAH and VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold:
            VAL = row['midpoint']
            break

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']

    # Calculate Value Area Width in percentage terms
    value_area_width = ((VAH - VAL) / VAH) * 100

    return pd.Series({'VAH': VAH, 'VAL': VAL, 'Value Area Width (%)': value_area_width})

result = df.groupby(df.index.date).apply(calculate_vah_val)
result'''


"import pandas as pd\n\n# Assuming resampled_df is already defined and contains the dataset\n#df = resampled_df.copy()\n#df = pd.DataFrame(resampled_df)\n\n#df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M:%S')\ndf.set_index('DateTime', inplace=True)\n\ndef calculate_vah_val(df_day):\n    total_volume = df_day['Volume'].sum()\n    volume_threshold = 0.7 * total_volume  # Here I have considered 70%\n\n    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculating the midpoint for each row\n    df_sorted = df_day.sort_values(by='midpoint', ascending=False)\n\n    cumulative_volume = 0\n    VAH = None\n    VAL = None\n\n    # Accumulate volume and find VAH and VAL\n    for index, row in df_sorted.iterrows():\n        cumulative_volume += row['Volume']\n        if cumulative_volume >= volume_threshold:\n            VAL = row['midpoint']\n            break\n\n    # The highest price in the sorted DataFrame is VAH\n    VAH = df_sorted.iloc[0

In [49]:



'''df_['DateTime'] = pd.to_datetime(df_['Date'] + ' ' + df_['Time'], format='%d-%m-%Y %H:%M:%S')
df_.set_index('DateTime', inplace=True)'''

'''def calculate_vah_val_poc(df_day):
    total_volume = df_day['Volume'].sum()
    volume_threshold = 0.7 * total_volume  # Here I have considered 70%

    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculating the midpoint for each row
    df_sorted = df_day.sort_values(by='midpoint', ascending=False)

    cumulative_volume = 0
    VAH = None
    VAL = None
    POC = None

    # Accumulate volume and find VAH and VAL
    for index, row in df_sorted.iterrows():
        cumulative_volume += row['Volume']
        if cumulative_volume >= volume_threshold and VAL is None:
            VAL = row['midpoint']
        if POC is None or row['Volume'] > POC:
            POC = row['midpoint']

    # The highest price in the sorted DataFrame is VAH
    VAH = df_sorted.iloc[0]['midpoint']

    # Calculate Value Area Width in percentage terms
    value_area_width = ((VAH - VAL) / VAH) * 100

    return pd.Series({'VAH': VAH, 'VAL': VAL, 'POC': POC, 'Value Area Width (%)': value_area_width})

# Calculate daily VAH, VAL, POC
daily_values = df1.groupby(df1.index.date).apply(calculate_vah_val_poc)

# Calculate rolling metrics
rolling_3d = daily_values[['VAH', 'VAL', 'POC']].rolling(window=3).mean()
rolling_5d = daily_values[['VAH', 'VAL', 'POC']].rolling(window=5).mean()

# Combine results
result = daily_values.join(rolling_3d, rsuffix='_3d').join(rolling_5d, rsuffix='_5d')
df1 = df1.merge(result, left_index=True, right_index=True, how='left')
df1
                                                              #forward fill if u dont want so many NaN
'''

"def calculate_vah_val_poc(df_day):\n    total_volume = df_day['Volume'].sum()\n    volume_threshold = 0.7 * total_volume  # Here I have considered 70%\n\n    df_day['midpoint'] = (df_day['High'] + df_day['Low']) / 2  # Calculating the midpoint for each row\n    df_sorted = df_day.sort_values(by='midpoint', ascending=False)\n\n    cumulative_volume = 0\n    VAH = None\n    VAL = None\n    POC = None\n\n    # Accumulate volume and find VAH and VAL\n    for index, row in df_sorted.iterrows():\n        cumulative_volume += row['Volume']\n        if cumulative_volume >= volume_threshold and VAL is None:\n            VAL = row['midpoint']\n        if POC is None or row['Volume'] > POC:\n            POC = row['midpoint']\n\n    # The highest price in the sorted DataFrame is VAH\n    VAH = df_sorted.iloc[0]['midpoint']\n    \n    # Calculate Value Area Width in percentage terms\n    value_area_width = ((VAH - VAL) / VAH) * 100\n    \n    return pd.Series({'VAH': VAH, 'VAL': VAL, 'POC': POC, '

In [16]:
import pandas as pd
import numpy as np

x = pd.DataFrame(resampled_df)

# Identifying Gap Up/Down
x['Prev_Close'] = x['Close'].shift(1)
x['Prev_High'] = x['High'].shift(1)
x['Prev_Low'] = x['Low'].shift(1)

# Identify gap up or gap down
x['Gap_Type'] = np.where(x['Open'] > x['Prev_High'], 'Gap Up',
                          np.where(x['Open'] < x['Prev_Low'], 'Gap Down', 'No Gap'))

# Determine if the gap was filled
x['Gap_Filled'] = np.where((x['Gap_Type'] == 'Gap Up') & (x['Low'] <= x['Prev_Close']), True,
                            np.where((x['Gap_Type'] == 'Gap Down') & (x['High'] >= x['Prev_Close']), True, False))

# Calculate percentage of gaps filled
total_gaps = x[x['Gap_Type'] != 'No Gap'].shape[0]
gaps_filled = x[x['Gap_Filled']].shape[0]
gap_fill_percentage = (gaps_filled / total_gaps) * 100 if total_gaps > 0 else 0

print(f'Percentage of gaps filled: {gap_fill_percentage:.2f}%')
x

#df[['Date', 'Open', 'High', 'Low', 'Close', 'Prev_Close', 'Gap_Type', 'Gap_Filled']]


Percentage of gaps filled: 61.21%


,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest,Prev_Close,Prev_High,Prev_Low,Gap_Type,Gap_Filled
0,BANKNIFTY_F2,01-01-2018,09:15:00,25634.00,25643.10,25608.0,25643.10,480.0,0.0,NaN,NaN,NaN,No Gap,False
1,BANKNIFTY_F2,01-01-2018,09:20:00,25631.50,25675.70,25631.5,25665.30,600.0,0.0,25643.10,25643.10,25608.00,No Gap,False
2,BANKNIFTY_F2,01-01-2018,09:25:00,25665.30,25665.30,25655.1,25655.10,240.0,0.0,25665.30,25675.70,25631.50,No Gap,False
3,BANKNIFTY_F2,01-01-2018,09:30:00,25647.70,25647.70,25647.7,25647.70,240.0,0.0,25655.10,25665.30,25655.10,Gap Down,False
4,BANKNIFTY_F2,01-01-2018,09:35:00,25647.70,25647.70,25647.0,25647.00,440.0,0.0,25647.70,25647.70,25647.70,No Gap,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542663,BANKNIFTY_F2,28-02-2023,15:10:00,40699.55,40699.55,40671.3,40691.00,1250.0,1170525.0,40700.00,40727.00,40670.85,No Gap,False
542664,BANKNIFTY_F2,28-02-2023,15:15:00,40700.00,40700.00,40670.0,40687.05,1850.0,1170650.0,40691.00,40699.55,40671.30,Gap Up,True
542665,BANKNIFTY_F2,28-02-2023,15:20:00,40696.30,40721.95,40695.0,40703.05,6000.0,1171475.0,40687.05,40700.00,40670.00,No Gap,False
542666,BANKNIFTY_F2,28-02-2023,15:25:00,40715.20,40715.20,40694.1,40706.00,4700.0,1173225.0,40703.05,40721.95,40695.00,No Gap,False


In [17]:
import pandas as pd

# Assuming resampled_df is already loaded as a DataFrame

# Ensure Date and Time columns are of type string
resampled_df['Date'] = resampled_df['Date'].astype(str)
resampled_df['Time'] = resampled_df['Time'].astype(str)

# Convert 'Date' and 'Time' to datetime
resampled_df['DateTime'] = pd.to_datetime(resampled_df['Date'] + ' ' + resampled_df['Time'], format='%d-%m-%Y %H:%M:%S')
resampled_df.set_index('DateTime', inplace=True)

# Remove duplicate Date and Time columns
resampled_df.drop(['Date', 'Time'], axis=1, inplace=True)

# Calculate previous day's close
resampled_df['PrevClose'] = resampled_df['Close'].shift(1)

# Identify gap up and gap down days
resampled_df['GapUp'] = (resampled_df['Open'] > resampled_df['PrevClose'])
resampled_df['GapDown'] = (resampled_df['Open'] < resampled_df['PrevClose'])

# Debug: Print the first few rows to verify the gap calculations
print(resampled_df[['Open', 'Close', 'PrevClose', 'GapUp', 'GapDown']].head(10))

# Resample to daily to calculate VAH and VAL
daily_df = resampled_df.resample('D').agg({
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()

# Calculate VAH and VAL (Value Area High/Low)
# Simplified method for VAH and VAL calculation
daily_df['VAH'] = daily_df['High'] - (daily_df['High'] - daily_df['Low']) * 0.382
daily_df['VAL'] = daily_df['Low'] + (daily_df['High'] - daily_df['Low']) * 0.382

# Debug: Print the daily_df to verify VAH and VAL calculations
print(daily_df.head(10))

# Merge VAH and VAL back to the original dataframe
resampled_df = resampled_df.merge(daily_df[['VAH', 'VAL']], left_index=True, right_index=True, how='left')

# Forward fill VAH and VAL for intraday data
resampled_df[['VAH', 'VAL']] = resampled_df[['VAH', 'VAL']].fillna(method='ffill')

# Check if the gap is above VAH or below VAL
resampled_df['GapAboveVAH'] = resampled_df['Open'] > resampled_df['VAH']
resampled_df['GapBelowVAL'] = resampled_df['Open'] < resampled_df['VAL']

# Calculate gap fill
resampled_df['GapFilled'] = resampled_df['Low'] <= resampled_df['PrevClose']

# Debug: Print the first few rows to verify gap above/below VAH/VAL and gap fill
print(resampled_df[['Open', 'VAH', 'VAL', 'GapAboveVAH', 'GapBelowVAL', 'GapFilled']].head(10))

# Resample to daily to count gap fills
daily_gaps = resampled_df.resample('D').agg({
    'GapUp': 'first',
    'GapDown': 'first',
    'GapAboveVAH': 'first',
    'GapBelowVAL': 'first',
    'GapFilled': 'max'
}).dropna()

# Debug: Print daily_gaps to verify the aggregation
print(daily_gaps.head(10))

# Calculate percentages
total_gap_ups = daily_gaps['GapUp'].sum()
total_gap_downs = daily_gaps['GapDown'].sum()
gap_ups_filled = daily_gaps[daily_gaps['GapUp'] & daily_gaps['GapFilled']].shape[0]
gap_downs_filled = daily_gaps[daily_gaps['GapDown'] & daily_gaps['GapFilled']].shape[0]

gap_up_fill_percentage = (gap_ups_filled / total_gap_ups) * 100 if total_gap_ups else 0
gap_down_fill_percentage = (gap_downs_filled / total_gap_downs) * 100 if total_gap_downs else 0

results = {
    "Total Gap Ups": total_gap_ups,
    "Total Gap Downs": total_gap_downs,
    "Gap Ups Filled": gap_ups_filled,
    "Gap Downs Filled": gap_downs_filled,
    "Gap Up Fill %": gap_up_fill_percentage,
    "Gap Down Fill %": gap_down_fill_percentage
}

print(results)


                        Open    Close  PrevClose  GapUp  GapDown
DateTime                                                        
2018-01-01 09:15:00  25634.0  25643.1        NaN  False    False
2018-01-01 09:20:00  25631.5  25665.3    25643.1  False     True
2018-01-01 09:25:00  25665.3  25655.1    25665.3  False    False
2018-01-01 09:30:00  25647.7  25647.7    25655.1  False     True
2018-01-01 09:35:00  25647.7  25647.0    25647.7  False    False
2018-01-01 09:40:00  25655.1  25655.0    25647.0   True    False
2018-01-01 09:45:00  25653.0  25650.0    25655.0  False     True
2018-01-01 09:50:00  25645.0  25650.0    25650.0  False     True
2018-01-01 09:55:00  25645.0  25650.0    25650.0  False     True
2018-01-01 10:00:00  25642.0  25642.0    25650.0  False     True
               High      Low    Close    Volume         VAH         VAL
DateTime                                                               
2018-01-01  25675.7  25481.3  25482.6   52720.0  25601.4392  25555.5608
2018